In [47]:
import torch
import pandas as pd
from dataset.party_dataset import ActiveSatelliteDataset, SatteliteLABELS

In [49]:
data_root = "C:/Users/XD278777/Desktop/worldstrat/dataset"

data = pd.read_csv(data_root + r'\metadata.csv')
# Rename 1st column to 'POI'
data = data.rename(columns={data.columns[0]: 'POI'})
data = data.loc[data['POI'] != 'ASMSpotter-1-1-1']
POI = data['POI'].unique()
dataset = ActiveSatelliteDataset(test_train_POI=POI, index=None)
print(dataset[0][0].shape)

torch.Size([12, 168, 168])


In [15]:
import torch
from transformers import AutoFeatureExtractor, AutoModel
from PIL import Image
import requests

# Load the feature extractor and the model
# feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base")


image = torch.randn(1, 3, 224, 224)

# Preprocess the image
# inputs = feature_extractor(images=image, return_tensors="pt")

# Extract features
with torch.no_grad():
    features = model(image).last_hidden_state

print(features.shape)

torch.Size([1, 257, 768])


In [16]:
(224/16)**2

196.0

In [50]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)
# print(image.shape)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs[0]
print(last_hidden_states.shape)


torch.Size([1, 257, 768])


c:\Users\XD278777\AppData\Local\miniconda3\envs\VFLAIR\lib\site-packages\torch\nn\functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(


In [66]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [104]:
import os
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoModel
import tifffile as tiff
import torchvision.transforms as transforms

image_path = r'C:\Users\XD278777\Desktop\worldstrat\dataset\lr_dataset\Amnesty POI-1-1-1\L2A\Amnesty POI-1-1-1-2-L2A_data.tiff'
image = tiff.imread(image_path)

In [106]:
image = transform(image).unsqueeze(0)

In [107]:
image.shape

torch.Size([1, 3, 168, 168])

In [112]:
model(image)[1].squeeze().shape

c:\Users\XD278777\AppData\Local\miniconda3\envs\VFLAIR\lib\site-packages\torch\nn\functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(


torch.Size([768])

In [1]:
import os
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoModel
import tifffile as tiff
import torchvision.transforms as transforms
from concurrent.futures import ThreadPoolExecutor
device = torch.device('cuda')

# Define a function to extract the class token from a batch of images
def extract_class_token_batch(image_paths, model, transform):
    images = []
    for image_path in image_paths:
        image = tiff.imread(image_path)
        image = transform(image)
        images.append(image)
    images = torch.stack(images).to(device)  # Stack and move to device (GPU)
    with torch.no_grad():
        features = model(images)
    class_tokens = features[1].squeeze().cpu().numpy()  # Assuming the class token is the first token
    return class_tokens

# Directory paths
input_dir = r'C:\Users\XD278777\Desktop\worldstrat\dataset\lr_dataset'  # Change to your input directory
output_dir = r'C:\Users\XD278777\Desktop\worldstrat\dataset\extracted_features'  # Change to your desired output directory

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load DINOv2 model
model = AutoModel.from_pretrained('facebook/dinov2-base').to(device)
model.eval()

# Transform definition
transform = transforms.Compose([
    transforms.Lambda(lambda image: torch.tensor(image.reshape(12, 168, 168)[2:5]))  # Reshape and select channels 2, 3, 4
])

# Collect all files
files_to_process = []
for root, _, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.tiff'):
            files_to_process.append(os.path.join(root, file))

# Batch size for processing images
batch_size = 512

# Function to save class tokens
def save_class_tokens(batch_file_paths, class_tokens, output_dir):
    for file_path, class_token in zip(batch_file_paths, class_tokens):
        relative_path = os.path.relpath(os.path.dirname(file_path), input_dir)
        output_path = os.path.join(output_dir, relative_path)
        os.makedirs(output_path, exist_ok=True)
        class_token_file_path = os.path.join(output_path, os.path.basename(file_path).replace('.tiff', '_class_token.npy'))
        np.save(class_token_file_path, class_token)

# Process files in batches with progress bar
with ThreadPoolExecutor() as executor:
    futures = []
    for i in tqdm(range(0, len(files_to_process), batch_size), desc="Processing images"):
        batch_file_paths = files_to_process[i:i+batch_size]
        class_tokens = extract_class_token_batch(batch_file_paths, model, transform)
        futures.append(executor.submit(save_class_tokens, batch_file_paths, class_tokens, output_dir))

    # Wait for all threads to complete
    for future in tqdm(futures, desc="Saving class tokens"):
        future.result()

print("Class token extraction completed.")



Processing images:   0%|          | 0/123 [00:00<?, ?it/s]c:\Users\XD278777\AppData\Local\miniconda3\envs\VFLAIR\lib\site-packages\torch\nn\functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
Saving class tokens: 100%|██████████| 123/123 [00:00<00:00, 159.72it/s]

Class token extraction completed.
